In [8]:
import numpy as np
import gdspy
from autograd import numpy as npa

################################################### design parameter #########################
resolution = 50  # Resolution
design_region_width = 3  # Design region width
design_region_height = 4  # Design region height
pml_size = 1  # PML region size
Sx = 1.0  # Height: from top to bottom PML 0 to LN, below that SiO2
Sy = 3.0  # Width: left and right PML, width 3um
Sz = 3.0  # Length (direction of propagation): 3
LNwg_h = 0.5  # LN waveguide
# Design space
design_region_x = LNwg_h
design_region_y = Sy
design_region_z = 2
Nx = int(resolution * design_region_x) + 1
Ny = int(resolution * design_region_y) + 1
Nz = int(resolution * design_region_z) + 1
##############################################################################################

# Load and process structure weight data
structure_weight = np.loadtxt('lastdesign.txt')  # Adjust path as needed
structure_weight = structure_weight.reshape(Nx, Ny * Nz)[Nx - 1]
data = npa.rot90(structure_weight.reshape(Ny, Nz))

# Check if the cell 'TOP' exists and delete it if it does
if 'TOP' in gdspy.current_library.cells:
    del gdspy.current_library.cells['TOP']

# Create the new cell
cell = gdspy.Cell('TOP')

# Rectangle dimensions
width = 1
height = 1
 
# Numpy 배열을 기반으로 GDS의 경로(Path) 생성
rectangles = []
for i in range(data.shape[0]):
    for j in range(data.shape[1]):
        if data[i, j] == 1:
            # 각 사각형의 좌표를 정밀하게 조정
            x0 = j * width
            y0 = -i * height
            x1 = (j + 1) * width
            y1 = -(i + 1) * height
            rectangle = gdspy.Rectangle((x0, y0), (x1, y1))
            rectangles.append(rectangle)

# Merge rectangles with precise alignment
if rectangles:
    merged_polygon = gdspy.boolean(rectangles, None, 'or', precision=1e-9)
    merged_polygon_1 = gdspy.boolean(merged_polygon, None, 'or', precision=1e-9)
    cell.add(merged_polygon_1)

# Set the unit and precision (optional)
# 1 micrometer (µm) is the default unit, and 1e-9 is the default precision (1 nanometer).
unit = 1e-6  # 1 micrometer per unit
precision = 1e-9  # 1 nanometer precision

# Save the GDS file with specified unit and precision
gdspy.write_gds('output.gds', unit=unit, precision=precision)

# Optionally, save an image of the cell as SVG.
cell.write_svg('output.svg')

# Display all cells using the internal viewer.
gdspy.LayoutViewer()



/tmp/ipykernel_1888830/169167960.py:70: DeprecationWarning: [GDSPY] Use of the global library is deprecated.  Pass LayoutViewer a GdsLibrary instance.
  gdspy.LayoutViewer()


<gdspy.viewer.LayoutViewer object .!layoutviewer>